In [1]:
!wget https://hgdownload.cse.ucsc.edu/goldenpath/hg38/chromosomes/chr21.fa.gz

--2024-03-26 13:12:17--  https://hgdownload.cse.ucsc.edu/goldenpath/hg38/chromosomes/chr21.fa.gz
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.198.53
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.198.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12709705 (12M) [application/x-gzip]
Saving to: ‘chr21.fa.gz’

chr21.fa.gz         100%[===================>]  12.12M  16.2MB/s    in 0.7s    

2024-03-26 13:12:18 (16.2 MB/s) - ‘chr21.fa.gz’ saved [12709705/12709705]



In [2]:
!wget https://hgdownload.cse.ucsc.edu/goldenpath/hg38/database/cpgIslandExt.txt.gz

--2024-03-26 13:12:18--  https://hgdownload.cse.ucsc.edu/goldenpath/hg38/database/cpgIslandExt.txt.gz
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.198.53
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.198.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 717984 (701K) [application/x-gzip]
Saving to: ‘cpgIslandExt.txt.gz’

cpgIslandExt.txt.gz 100%[===================>] 701.16K  1.85MB/s    in 0.4s    

2024-03-26 13:12:19 (1.85 MB/s) - ‘cpgIslandExt.txt.gz’ saved [717984/717984]



In [3]:
!gunzip /content/cpgIslandExt.txt.gz

In [4]:
!gunzip /content/chr21.fa.gz

In [5]:
input_file_path = '/content/cpgIslandExt.txt'
output_file_path = '/content/chr21.txt'

delimiter = '\t'

with open(input_file_path, 'r') as input_file, open(output_file_path, 'w') as output_file:
    for line in input_file:
        columns = line.strip().split(delimiter)
        if len(columns) > 1 and columns[1] == 'chr21':
            output_file.write(line)

In [6]:
!grep ">" /content/chr21.fa

>chr21


In [7]:
import numpy as np

In [8]:
def get_seq(seqfile):
    seq = ''
    with open(seqfile) as f:
        for line in f:
            if line[0] == '>': continue
            seq += line.rstrip()
    return seq

In [9]:
seq = get_seq("/content/chr21.fa")

In [10]:
def get_ranges(bedfile, start_col=2, end_col=3):
    ranges = []
    with open(bedfile) as f:
        for line in f:
            line_list = line.split()
            ranges.append((int(line_list[start_col]), int(line_list[end_col])))
    return ranges

In [11]:
ranges = get_ranges('/content/chr21.txt')

In [12]:
def get_states(ranges, seq):
    state = np.zeros(len(seq)+1)
    for i, j in ranges:
        state[i:j+1] = 1
    return state[1:]

In [13]:
state = get_states(ranges, seq)

In [14]:
def list_to_str(state_list):
    end = ''
    for num in state_list:
        if num:
            end += 'Y'
        else:
            end += 'N'
    return end

In [15]:
state = list_to_str(state)

In [16]:
def get_matrices(seq, state, nuc='ACGTN', states='NY'):
    tm_dict = dict()
    ep_dict = dict()
    seq = seq.upper()

    for st1, st2, ch1 in zip(state, state[1:], seq):
        tm_dict[st1+st2] = tm_dict.get(st1+st2, 0) + 1
        ep_dict[st1+ch1] = ep_dict.get(st1+ch1, 0) + 1
    return tm_dict, ep_dict

In [17]:
from collections import Counter

def get_matrices_optimized(seq, state):
    seq = seq.upper()

    transition_pairs = zip(state, state[1:])
    emission_pairs = zip(state, seq)

    tm_dict = Counter(transition_pairs)
    ep_dict = Counter(emission_pairs)

    tm_dict = {k[0] + k[1]: v for k, v in tm_dict.items()}
    ep_dict = {k[0] + k[1]: v for k, v in ep_dict.items()}

    return tm_dict, ep_dict

In [18]:
%%time
tm_dict, ep_dict = get_matrices_optimized(seq, state)

CPU times: user 15.4 s, sys: 154 ms, total: 15.5 s
Wall time: 15.7 s


In [19]:
%%time
tm_dict, ep_dict = get_matrices(seq, state)

CPU times: user 32.3 s, sys: 146 ms, total: 32.5 s
Wall time: 32.7 s


In [20]:
import pandas as pd

In [21]:
expanded_data = [(key[0], key[1], value) for key, value in ep_dict.items()]

df = pd.DataFrame(expanded_data, columns=['Index', 'Column', 'Value'])

pivot_df = df.pivot(index='Index', columns='Column', values='Value')

In [22]:
pivot_df

Column,A,C,G,N,T
Index,,,,,
N,11769906.0,8063026.0,8106230.0,6621363.0,11800527.0
Y,50758.0,122218.0,120151.0,NaN,55803.0


In [23]:
normalized_df = pivot_df.div(pivot_df.sum(axis=1), axis=0)
ep_dict_df = normalized_df
ep_dict_df

Column,A,C,G,N,T
Index,,,,,
N,0.253875,0.173918,0.174850,0.142822,0.254535
Y,0.145468,0.350265,0.344341,NaN,0.159926


In [24]:
import pandas as pd

data = tm_dict

# Creating a nested dictionary to properly align the data for the DataFrame
nested_dict = {}
for key, value in data.items():
    row_key, col_key = key[0], key[1]
    if row_key not in nested_dict:
        nested_dict[row_key] = {}
    nested_dict[row_key][col_key] = value

# Creating the DataFrame from the nested dictionary
df = pd.DataFrame.from_dict(nested_dict, orient='index')
# Calculating the probability of each row by dividing each value by the row sum
row_probabilities = df.div(df.sum(axis=1), axis=0)
tm_dict_df = row_probabilities
tm_dict_df

,N,Y
N,0.999990,0.000010
Y,0.001278,0.998722


In [25]:
ep_dict_df

Column,A,C,G,N,T
Index,,,,,
N,0.253875,0.173918,0.174850,0.142822,0.254535
Y,0.145468,0.350265,0.344341,NaN,0.159926


In [26]:
tm_dict_df

,N,Y
N,0.999990,0.000010
Y,0.001278,0.998722


In [28]:
def hmm_forward(seq, tm, ep, begin_prob=[0.5, 0.5], end_prob=[0.5, 0.5]):
    nuc = ep.columns
    seq_len = len(seq)
    states = tm.columns
    states_len = len(states)
    forward = np.zeros((states_len, seq_len))
    fp = 0.0

    for i, state in enumerate(states):
        forward[i][0] = begin_prob[i] * ep[seq[0]][state]

    for i in range(len(seq)):
        for j, first_st in enumerate(states):
            for k, second_st in enumerate(states):
                forward[j][i] += forward[k][i-1] * tm[second_st][first_st] * ep[seq[i]][first_st]

    for k, second_state in enumerate(states):
        fp += forward[k][-1]

    return fp

In [29]:
seq = 'ACTTGGCA'

In [30]:
ep_dict_df[seq[0]]['N']

0.25387486893092936

In [31]:
hmm_forward('ACTTGGCA', tm_dict_df, ep_dict_df).tolist()

5.8730338195669305e-06

In [ ]:
begin_prob[i] * ep[state][nuc[seq[0]]]

In [ ]:
nuc = ep_dict_df.columns
seq_len = len(seq)
states = tm_dict_df.columns
states_len = len(states)
seq_len, states_len

In [ ]:
nuc, states